<a href="https://colab.research.google.com/github/UzunDemir/10_Hybrid_Recommender_Systems/blob/main/10_Hybrid_Recommender_Systems.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10.2 Практическая работа
## Цель практической работы

Научиться сочетать контентный и коллаборативный подходы для построения рекомендательной системы.

## Что входит в работу
1. Создать user-item-матрицу, разбить данные на тест и контроль.
2. Применить метод матричной факторизации и собрать признаки для контентной модели.
3. Применить комбинацию методов, подсчитать метрики.


## Задание 1. Создание user-item-матрицы, разбиение данных на тест и контроль
### Что нужно сделать
1. Постройте user-item-матрицу на основе данных о прочитанных книгах из таблицы ratings.csv.

2. Подключите библиотеку Implicit. Для работы с библиотекой переведите данные рейтинга в бинарные оценки: 1 — книга понравилась (получила оценку 4 или 5), 0 — книга не понравилась.
Для дальнейшей оценки качества разбейте данные на тест и контроль. Так как в данных нет настоящей даты прочтения, воспользуйтесь обходным способом:
* пронумеруйте прочитанные книги каждого пользователя в порядке их появления;
* переведите номера прочитанных книг в доли по формуле: порядковый номер / общее количество прочитанных пользователем книг.
3. Определите, какое количество данных оставите на обучение, какое — на контроль. Например, 70% книг каждого пользователя — на обучение.
4. На основе данных для обучения постройте user-item-матрицу.

In [2]:
!pip install implicit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 35.3 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import implicit

from sklearn.model_selection import train_test_split
from pandas.api.types import CategoricalDtype
from xgboost import XGBClassifier

In [4]:
# Функция для рассчета метрики AP@K

def apk_score(preds, actual, k=10):
    preds = preds[:k]  # Оставляем только первые k предсказаний
    precisions_at_k = []
    mask = []  # Вспомогательный вектор, определяющий релевантность предсказаний

    for i in range(k):
        # Создаем маску: 1, если предсказание релевантно, иначе 0
        mask = [1 if p in actual else 0 for p in preds[:i+1]]
        # Вычисляем точность на уровне k
        precisions_at_k.append(sum(mask) * mask[-1] / (i+1))

    n_hits = sum(mask)  # Количество релевантных элементов
    if n_hits == 0:
        n_hits = 1  # Избегаем деления на ноль

    return sum(precisions_at_k) / n_hits  # Возвращаем среднюю точность


### Загрузка данных

In [5]:
import gdown

# URL файла
url = "https://drive.google.com/uc?id=1BLR_z7h2OQ6ISUVLjVApWLeqRR8EDUAS"

# Путь для сохранения файла
output = "ratings.csv"

gdown.download(url, output, quiet=False)


# Чтение CSV файла
rating_df = pd.read_csv("ratings.csv")



print(rating_df.shape)
print('unique user_id', rating_df.user_id.nunique())
print('unique book_id', rating_df.book_id.nunique())
# Просмотр первых строк
rating_df.head()

Downloading...
From: https://drive.google.com/uc?id=1BLR_z7h2OQ6ISUVLjVApWLeqRR8EDUAS
To: /content/ratings.csv
100%|██████████| 72.1M/72.1M [00:01<00:00, 55.9MB/s]


(5976479, 3)
unique user_id 53424
unique book_id 10000


,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [ ]:
books_df = pd.read_csv('goodbooks-10k-master/books.csv', usecols=['book_id', 'goodreads_book_id', 'title'])
print(books_df.shape)
print('unique book_id', books_df.book_id.nunique())
books_df.head()

(10000, 3)
unique book_id 10000


,book_id,goodreads_book_id,title
0,1,2767052,"The Hunger Games (The Hunger Games, #1)"
1,2,3,Harry Potter and the Sorcerer's Stone (Harry P...
2,3,41865,"Twilight (Twilight, #1)"
3,4,2657,To Kill a Mockingbird
4,5,4671,The Great Gatsby


In [8]:
import requests
import gzip
import shutil

# URL файла
url = 'https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/goodreads_book_genres_initial.json.gz'

# Скачивание файла
response = requests.get(url, stream=True)
with open('/content/goodreads_book_genres_initial.json.gz', 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)
del response

# Распаковка файла
with gzip.open('/content/goodreads_book_genres_initial.json.gz', 'rb') as f_in:
    with open('/content/goodreads_book_genres_initial.json', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Проверка содержимого директории
!ls -l /content/


total 290232
-rw-r--r-- 1 root root    895195 Sep 24 14:12 goodbooks-10k.csv
-rw-r--r-- 1 root root 199903667 Sep 24 14:15 goodreads_book_genres_initial.json
-rw-r--r-- 1 root root  24253992 Sep 24 14:15 goodreads_book_genres_initial.json.gz
-rw-r--r-- 1 root root  72126826 Jun 13 08:34 ratings.csv
drwxr-xr-x 1 root root      4096 Sep 20 13:22 sample_data


In [9]:
genres_df = pd.read_json('goodreads_book_genres_initial.json', lines=True)
print(genres_df.shape)
genres_df.head()

(2360655, 2)


,book_id,genres
0,5333265,"{'history, historical fiction, biography': 1}"
1,1333909,"{'fiction': 219, 'history, historical fiction,..."
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys..."
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri..."
4,287140,{'non-fiction': 3}


https://github.com/zygmuntz/goodbooks-10k/blob/master/books.csv

In [11]:
# URL файла
url = "https://github.com/zygmuntz/goodbooks-10k/blob/master/books.csv"

# Путь для сохранения файла
output = "books.csv"

gdown.download(url, output, quiet=False)


Downloading...
From: https://github.com/zygmuntz/goodbooks-10k/blob/master/books.csv
To: /content/books_df.csv
300kB [00:00, 95.5MB/s]


'books_df.csv'

In [14]:
# Чтение CSV файла
books_df = pd.read_csv("/content/books.csv")



print(books_df.shape)
print('unique book_id', books_df.book_id.nunique())
books_df.head()

(10000, 23)
unique book_id 10000


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


### Предобработка данных

In [15]:
rating_df['rating'] = rating_df['rating'].apply(lambda x: 1 if x > 3 else 0)
rating_df.head()

,user_id,book_id,rating
0,1,258,1
1,2,4081,1
2,2,260,1
3,2,9296,1
4,2,2318,0


In [16]:
# Присвоение порядкового номера прочтенным книгам для каждого пользователя.

user_dict = {str(user): 0 for user in rating_df.user_id.unique()} # Словарь для хранения порядковых номеров книг для каждого пользователя (для ускорения)

def book_order_st1(row):
    user_dict[f'{row.user_id}'] += 1
    return user_dict[f'{row.user_id}']

In [17]:
def book_order_st2(user_id, order):
    order = order / user_dict[f'{user_id}']
    return order

vbook_order_st2 = np.vectorize(book_order_st2)

In [18]:
rating_df['book_order'] = rating_df.apply(book_order_st1, axis=1)

In [19]:
rating_df['book_order'] = vbook_order_st2(rating_df.user_id, rating_df.book_order)

In [20]:
rating_df[rating_df['user_id']==1]

,user_id,book_id,rating,book_order
0,1,258,1,0.008547
75,1,268,0,0.017094
76,1,5556,0,0.025641
77,1,3638,0,0.034188
78,1,1796,1,0.042735
...,...,...,...,...
5704475,1,142,1,0.965812
5704476,1,642,1,0.974359
5704477,1,901,1,0.982906
5704479,1,212,0,0.991453


In [21]:
# Разделение данных на тренировочную и тестовую выборки.

train = rating_df[rating_df['book_order'] <= 0.7]
test = rating_df[rating_df['book_order'] > 0.7]

In [22]:
# Признаки о пользователях: средний рейтинг, который ставит пользователь
user_features = train.groupby('user_id')['rating'].mean().reset_index()
user_features.columns = ['user_id', 'user_avg_rating']

# Объединение признаков с основным датасетом
train = train.merge(user_features, on='user_id', how='left')

In [23]:
def simple_one_hot(genre_dict, genre):
    if genre in genre_dict:
        return 1
    return 0

In [24]:
genres_df = genres_df[genres_df.book_id.isin(books_df.goodreads_book_id)]
genres_df.columns = ['book_id', 'genres_dict']

all_genres = set()
for dict_genre in genres_df.genres_dict:
    for elem in list(dict_genre.keys()):
        all_genres.add(elem)

for genre in all_genres:
    genres_df[genre] = 0

for genre in all_genres:
    genres_df[genre] = genres_df.apply(lambda df: simple_one_hot(df['genres_dict'], genre), axis=1)

genres_df = genres_df.drop(columns='genres_dict')

genres_df.head()

<ipython-input-24-f43341af834b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genres_df[genre] = 0
<ipython-input-24-f43341af834b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genres_df[genre] = 0
<ipython-input-24-f43341af834b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-v

,book_id,"history, historical fiction, biography","fantasy, paranormal",non-fiction,children,"mystery, thriller, crime",young-adult,poetry,"comics, graphic",romance,fiction
3,6066819,0,0,0,0,1,0,0,0,1,1
15,89375,1,0,1,0,0,0,0,1,0,1
583,54270,1,0,1,0,0,0,0,1,0,0
807,38568,0,1,0,0,0,0,0,0,1,1
816,38562,0,1,0,0,1,0,0,0,1,1


In [25]:
train = train.merge(books_df[['book_id', 'goodreads_book_id']], left_on='book_id', right_on='book_id', how='left')
train = train.merge(genres_df, left_on='goodreads_book_id', right_on='book_id', how='left')

In [26]:
users_profiles = train.groupby('user_id')[list(all_genres)].sum()
users_profiles.columns = ['user_'+name for name in list(users_profiles)]
train = train.merge(users_profiles, left_on='user_id', right_on='user_id', how='left')

In [27]:
train.columns = ['book_'+item if item in all_genres else item for item in list(train)]

In [ ]:
# Использованы векторные представления, полученные в ходе выполнения модуля 8.4

# w2v_emb_titles = np.load('w2v_title_vecs.npy')
# emb_titles_cols = [f'titles_{x}' for x in range(np.array(w2v_emb_titles).shape[1])]

In [1]:
# books_df = pd.concat([books_df, pd.DataFrame(columns=emb_titles_cols, data=w2v_emb_titles)], axis=1)

In [29]:
train = train.merge(books_df, left_on='book_id_x', right_on='book_id')

In [30]:
train.fillna(0, inplace=True)

### Подготовка user-item матрицы

In [31]:
user_idx = train['user_id'].unique()
book_idx = train['book_id'].unique()

rows = train['user_id'].astype(CategoricalDtype(categories=user_idx)).cat.codes
cols = train['book_id'].astype(CategoricalDtype(categories=book_idx)).cat.codes

In [32]:
matrix = sparse.csr_matrix((train['rating'], (rows, cols)), shape=(len(user_idx), len(book_idx)))
matrix = matrix.toarray()
matrix.shape

(53424, 10000)

## Задание 2. Применение метода матричной факторизации и сбор признаков для контентной модели
### Что нужно сделать
1. Рассчитайте baseline, с которым будете сравнивать результаты.

2. Реализуйте функцию, которая подсчитывает метрику AP@K. На вход функция принимает список рекомендаций и список книг, положительно оценённых пользователем, то есть с оценкой 4 или 5.
3. Обучите алгоритм ALS из библиотеки Implicit с базовыми параметрами.
Подсчитайте метрику mAP@10 для алгоритма ALS, предварительно отделив N случайных юзеров для тестирования.

In [33]:
# Случайная подвыборка из 500 уникальных пользователей.
user_sample = pd.DataFrame(train.user_id.unique()).sample(n=500, random_state=42)
user_sample = user_sample[0].values

# Создание подвыборки из тестовой выборки для оценки.
test_sample = test[test['user_id'].isin(user_sample)]

In [34]:
model = implicit.als.AlternatingLeastSquares()
model.fit(sparse.csr_matrix(matrix))

/usr/local/lib/python3.10/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

In [35]:
preds = model.recommend(user_sample, sparse.csr_matrix(matrix)[user_sample])[0]

In [36]:
scores = []
for user, rec in zip(user_sample, preds):
    actual = test_sample[(test_sample.user_id == user) & (test_sample.rating==1)].book_id.tolist()
    score = apk_score(rec, actual, k=10)
    scores.append(score)

print('baseline ALS mAP@10: ', sum(scores) / len(scores))

baseline ALS mAP@10:  0.023709523809523802


## Задание 3. Применение комбинации методов, подсчёт метрик
### Что нужно сделать
Добавьте контентные данные для финального ранжирования результатов.

1. Для данных из train-части соберите признаки, например интересы, векторные представления текстов из title и так далее. Обучите любой алгоритм классификации, например Random Forest или CatBoost. Сохраните признаки пользователей и книг в отдельные таблицы.
2. С помощью метода ALS получите до 30 рекомендаций для каждого из пользователей.
Проранжируйте книги-кандидаты из метода ALS с помощью классификатора и выберите топ-10 книг по вероятности.
3. Подсчитайте mAP@10 для гибридных рекомендаций. Сделайте выводы о качестве полученной модели относительно обычного ALS. Укажите, какой подход работает дольше.

In [37]:
cols_for_using = [
 'book_mystery, thriller, crime',
 'book_non-fiction',
 'book_romance',
 'book_fantasy, paranormal',
 'book_poetry',
 'book_fiction',
 'book_young-adult',
 'book_history, historical fiction, biography',
 'book_comics, graphic',
 'book_children',
 'user_mystery, thriller, crime',
 'user_non-fiction',
 'user_romance',
 'user_fantasy, paranormal',
 'user_poetry',
 'user_fiction',
 'user_young-adult',
 'user_history, historical fiction, biography',
 'user_comics, graphic',
 'user_children',
 'user_avg_rating']

In [38]:
test = test.merge(user_features, on='user_id', how='left')

test = test.merge(books_df[['book_id', 'goodreads_book_id']], left_on='book_id', right_on='book_id', how='left')
test = test.merge(genres_df, left_on='goodreads_book_id', right_on='book_id', how='left')

test.columns = ['book_'+item if item in all_genres else item for item in list(test)]

test = test.merge(users_profiles, left_on='user_id', right_on='user_id', how='left')

test = test.merge(books_df, left_on='book_id_x', right_on='book_id')

test.fillna(0, inplace=True)
test[cols_for_using] = test[cols_for_using].astype('float32')

In [39]:
train.shape

(4159553, 50)

#### Обучение

In [40]:
params_dict = {
        'seed': 42,
        'learning_rate': 0.1,
        'n_estimators': 1000,
        'max_depth': 6,
        'min_child_weight': 5,
        'gamma': 1,
        'device': 'cuda',
        'tree_method': 'hist',
        'eval_metric': 'auc',
        'early_stopping_rounds': 50,
        'verbosity': 0
    }

In [41]:
xgb_model = XGBClassifier(**params_dict)

In [44]:
xgb_model.fit(train[cols_for_using],
          train['rating'],
          eval_set=[(test[cols_for_using], test['rating'])],
          verbose=50)

[0]	validation_0-auc:0.69506
[50]	validation_0-auc:0.69965
[100]	validation_0-auc:0.70227
[150]	validation_0-auc:0.70336
[200]	validation_0-auc:0.70376
[224]	validation_0-auc:0.70376


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=50,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=1, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [45]:
als_model = implicit.als.AlternatingLeastSquares(factors=26, iterations=30, random_state=42)
als_model.fit(sparse.csr_matrix(matrix))

  0%|          | 0/30 [00:00<?, ?it/s]

#### Оценка

In [46]:
user_sample

array([ 3078, 50604, 35154,  6279,  4666, 32483, 23086,  3335, 11891,
       33736, 18323, 20807, 12816, 45281, 24315, 17449,   768, 27297,
       15554,  3012,  6871,  5190, 32063, 47795,  9330, 41847, 37008,
       29564, 28050, 37714, 19290, 31019, 46001, 53177, 10841, 50230,
       29121, 47088, 11940,  3692,  6114, 22499,  5209, 31229,  2063,
        1718, 37721,  1484, 19517, 51143, 24152, 30871, 20169, 30267,
       31668,  2479, 17624, 14981, 33555, 44031, 29770, 53272, 42351,
       27883, 12058, 39656, 12221, 35531, 37570, 22794, 20407, 39745,
       16358, 51576, 34744,  3412, 13015, 10387,  4488, 23300, 35679,
       36710, 39381, 42168,  4775, 49185, 19924,  1088, 49880, 40114,
       10500, 32465,  7699, 14884, 29217, 40784, 42117, 24451, 29239,
        6989, 17406, 38014,  8722, 50970, 29212, 34052, 45088, 28391,
       43937, 30494,  7045, 34063, 12644, 48081, 52804,   345, 29018,
       19828, 25435, 33019,  5499, 42834, 34503, 19635,  8739,  7415,
       49394, 13742,

In [47]:
als_preds = als_model.recommend(user_sample, sparse.csr_matrix(matrix)[user_sample], N=30)[0]

In [48]:
als_preds

array([[6026, 1286,   19, ...,   86,  300,  596],
       [7230, 7522,  384, ..., 7504, 2895, 7006],
       [1890, 5874, 3052, ..., 2661, 3051, 6368],
       ...,
       [ 484,  515,  516, ...,   32,  663,   69],
       [ 591,  262, 1100, ...,  258, 1429,  961],
       [3545, 5985,  456, ...,  729,  357, 7015]], dtype=int32)

In [49]:
user_recs_dict = {user_id: recs for user_id, recs in zip(user_sample, als_preds)}

In [50]:
test_sample_df = pd.DataFrame(columns=['user_id', 'book_id'])
for user_id, recom in user_recs_dict.items():
    user_df = pd.DataFrame({'user_id': [user_id for i in range(len(recom))], 'book_id': recom})
    test_sample_df = pd.concat([test_sample_df, user_df], axis=0)

In [51]:
print(test_sample_df.shape)
test_sample_df.head()

(15000, 2)


,user_id,book_id
0,3078,6026
1,3078,1286
2,3078,19
3,3078,488
4,3078,938


In [53]:
test_sample_df = test_sample_df.merge(user_features, on='user_id', how='left')

test_sample_df = test_sample_df.merge(books_df[['book_id', 'goodreads_book_id']], left_on='book_id', right_on='book_id', how='left')
test_sample_df = test_sample_df.merge(genres_df, left_on='goodreads_book_id', right_on='book_id', how='left')

test_sample_df.columns = ['book_'+item if item in all_genres else item for item in list(test_sample_df)]

test_sample_df = test_sample_df.merge(users_profiles, left_on='user_id', right_on='user_id', how='left')

test_sample_df = test_sample_df.merge(books_df, left_on='book_id_x', right_on='book_id', how='left')

test_sample_df.fillna(0, inplace=True)
test_sample_df[cols_for_using] = test_sample_df[cols_for_using].astype('float32')
test_sample_df.shape

(15000, 48)

In [55]:
xgb_preds = xgb_model.predict_proba(test_sample_df[cols_for_using])[:, 1]

In [56]:
test_sample_df['rank'] = xgb_preds

In [57]:
test_sample_df = test_sample_df[['user_id', 'book_id_x', 'rank']]

In [58]:
test_sample_df.head()

,user_id,book_id_x,rank
0,3078,6026,0.475324
1,3078,1286,0.738599
2,3078,19,0.666785
3,3078,488,0.609427
4,3078,938,0.475324


In [59]:
test_sample_sorted = test_sample_df.groupby('user_id').apply(lambda x: x.sort_values('rank', ascending=False)).reset_index(drop=True)

In [60]:
scores = []
for user in test_sample_sorted.user_id.unique():
    recs = test_sample_sorted[test_sample_sorted['user_id'] == user].book_id_x.values
    recs = recs[:10]

    actual = test_sample[(test_sample.user_id == user) & (test_sample.rating==1)].book_id.tolist()
    score = apk_score(recs, actual, k=10)
    scores.append(score)

print('Итоговый показатель mAP@10: ', sum(scores) / len(scores))

Итоговый показатель mAP@10:  0.03332023809523809


Комбинированный подход оказался незначительно эффективнее с точки зрения метрики качества, однако затратнее по времени (за счет необходимости в более сложной предобработке данных и обучения дополнительной модели)

### Пайплайн (Дополнительно)

In [61]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin

In [63]:
rating_df = pd.read_csv('ratings.csv')
print(rating_df.shape)
print('unique user_id', rating_df.user_id.nunique())
print('unique book_id', rating_df.book_id.nunique())
rating_df.head()

(5976479, 3)
unique user_id 53424
unique book_id 10000


,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [64]:
books_df = pd.read_csv('books.csv', usecols=['book_id', 'goodreads_book_id', 'title'])
print(books_df.shape)
print('unique book_id', books_df.book_id.nunique())
books_df.head()

(10000, 3)
unique book_id 10000


,book_id,goodreads_book_id,title
0,1,2767052,"The Hunger Games (The Hunger Games, #1)"
1,2,3,Harry Potter and the Sorcerer's Stone (Harry P...
2,3,41865,"Twilight (Twilight, #1)"
3,4,2657,To Kill a Mockingbird
4,5,4671,The Great Gatsby


In [65]:
genres_df = pd.read_json('goodreads_book_genres_initial.json', lines=True)
print(genres_df.shape)
genres_df.head()

(2360655, 2)


,book_id,genres
0,5333265,"{'history, historical fiction, biography': 1}"
1,1333909,"{'fiction': 219, 'history, historical fiction,..."
2,7327624,"{'fantasy, paranormal': 31, 'fiction': 8, 'mys..."
3,6066819,"{'fiction': 555, 'romance': 23, 'mystery, thri..."
4,287140,{'non-fiction': 3}


In [66]:
rating_df['rating'] = rating_df['rating'].apply(lambda x: 1 if x > 3 else 0)

In [67]:
user_dict = {str(user): 0 for user in rating_df.user_id.unique()} # Словарь для хранения порядковых номеров книг для каждого пользователя (для ускорения)

def book_order_st1(row):
    user_dict[f'{row.user_id}'] += 1
    return user_dict[f'{row.user_id}']

In [68]:
def book_order_st2(user_id, order):
    order = order / user_dict[f'{user_id}']
    return order

vbook_order_st2 = np.vectorize(book_order_st2)

In [69]:
rating_df['book_order'] = rating_df.apply(book_order_st1, axis=1)
rating_df['book_order'] = vbook_order_st2(rating_df.user_id, rating_df.book_order)

In [70]:
train = rating_df[rating_df['book_order'] <= 0.7]
test = rating_df[rating_df['book_order'] > 0.7]

In [71]:
# Случайная подвыборка из 500 уникальных пользователей.
user_sample = pd.DataFrame(train.user_id.unique()).sample(n=500, random_state=42)
user_sample = user_sample[0].values

# Создание подвыборки из тестовой выборки для оценки.
test_sample = test[test['user_id'].isin(user_sample)]

In [72]:
cols_for_using = [
 'book_mystery, thriller, crime',
 'book_non-fiction',
 'book_romance',
 'book_fantasy, paranormal',
 'book_poetry',
 'book_fiction',
 'book_young-adult',
 'book_history, historical fiction, biography',
 'book_comics, graphic',
 'book_children',
 'user_mystery, thriller, crime',
 'user_non-fiction',
 'user_romance',
 'user_fantasy, paranormal',
 'user_poetry',
 'user_fiction',
 'user_young-adult',
 'user_history, historical fiction, biography',
 'user_comics, graphic',
 'user_children',
 'user_avg_rating']

In [73]:
# При вызове fit формирует user-item матрицу на основе тренировочных данных и обучает als-модель
# При вызове transform формирует для каждого уникального пользователя список из 30 рекомендаций, возвращает pd.DataFrame с парами пользователь - рекомендованная книга.
class AlsFilter(BaseEstimator, TransformerMixin):
    def __init__(self, als_params):
        self.als_params = als_params
        self.als_model = implicit.als.AlternatingLeastSquares(**self.als_params)

    def fit(self, x, y=None):
        user_idx = x['user_id'].unique()
        book_idx = x['book_id'].unique()

        rows = x['user_id'].astype(CategoricalDtype(categories=user_idx)).cat.codes
        cols = x['book_id'].astype(CategoricalDtype(categories=book_idx)).cat.codes

        self.matrix = sparse.csr_matrix((y, (rows, cols)), shape=(len(user_idx), len(book_idx)))

        self.als_model.fit(self.matrix)
        return self

    def transform(self, x):
        als_preds = self.als_model.recommend(x, self.matrix[x], N=30)[0]
        user_recs_dict = {user_id: recs for user_id, recs in zip(x, als_preds)}

        test_sample_df = pd.DataFrame(columns=['user_id', 'book_id'])
        for user_id, recom in user_recs_dict.items():
            user_df = pd.DataFrame({'user_id': [user_id for i in range(len(recom))], 'book_id': recom})
            test_sample_df = pd.concat([test_sample_df, user_df], axis=0)

        return test_sample_df

    def fit_transform(self, x, y=None):
        self.fit(x, y)
        return pd.concat([x, y], axis=1)


In [78]:
# Трансформер производит все преобразования, связанные с контентными данными.
class ContentPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, books, genres):
        self.books = books.copy()
        self.genres = genres.copy()
        self.users_profiles = pd.DataFrame()
        self.all_genres = set()
        self.emb_titles_cols = []

    def simple_one_hot(self, genre_dict, genre):
        if genre in genre_dict:
            return 1
        return 0

    def fit(self, x, y=None):
        # Средняя оценка, поставленная пользователем.
        self.user_features = x.groupby('user_id')['rating'].mean().reset_index()
        self.user_features = x.groupby('user_id')['rating'].mean().reset_index()
        self.user_features.columns = ['user_id', 'user_avg_rating']

        # Жанровая принадлежность книг.
        self.genres = self.genres[self.genres.book_id.isin(self.books.goodreads_book_id)]
        self.genres.columns = ['book_id', 'genres_dict']


        for dict_genre in self.genres.genres_dict:
            for elem in list(dict_genre.keys()):
                self.all_genres.add(elem)

        for genre in self.all_genres:
            self.genres[genre] = 0

        for genre in self.all_genres:
            self.genres[genre] = self.genres.apply(lambda df: self.simple_one_hot(df['genres_dict'], genre), axis=1)

        self.genres = self.genres.drop(columns='genres_dict')

        x = x.merge(self.books[['book_id', 'goodreads_book_id']], left_on='book_id', right_on='book_id', how='left')
        x = x.merge(self.genres, left_on='goodreads_book_id', right_on='book_id', how='left')

        # Жанровые предпочтения пользователей.
        self.users_profiles = x.groupby('user_id')[list(self.all_genres)].sum()
        self.users_profiles.columns = ['user_'+name for name in list(self.users_profiles)]

        # Эмбеддинги заголовков.
        # w2v_emb_titles = np.load('w2v_title_vecs.npy')
        # self.emb_titles_cols = [f'titles_{x}' for x in range(np.array(w2v_emb_titles).shape[1])]
        # self.books = pd.concat([self.books, pd.DataFrame(columns=self.emb_titles_cols, data=w2v_emb_titles)], axis=1)

        return self

    def transform(self, x):
        x = x.merge(self.user_features, on='user_id', how='left')
        x = x.merge(self.books[['book_id', 'goodreads_book_id']], left_on='book_id', right_on='book_id', how='left')
        x = x.merge(self.genres, left_on='goodreads_book_id', right_on='book_id', how='left')
        x = x.merge(self.users_profiles, left_on='user_id', right_on='user_id', how='left')
        x.columns = ['book_'+item if item in self.all_genres else item for item in list(x)]
        x = x.merge(self.books, left_on='book_id_x', right_on='book_id')
        x.fillna(0, inplace=True)
        return x[['user_id', 'book_id_x'] + cols_for_using + self.emb_titles_cols]

In [79]:
# Модель возвращает pd.DataFrame с рекомендациями для каждого пользователя. Рекомендации ранжированы и отсортированы для каждого пользователя.
class XGBRecomendation(BaseEstimator, ClassifierMixin):
    def __init__(self, xgb_params):
        self.xgb_params = xgb_params
        self.xgb_model = XGBClassifier(**self.xgb_params)

    def fit(self, x, y):
        self.xgb_model.fit(x.drop(columns=['user_id', 'book_id_x']), y)
        return self

    def predict(self, x):
        xgb_preds = self.xgb_model.predict_proba(x.drop(columns=['user_id', 'book_id_x']))[:, 1]
        x['rank'] = xgb_preds
        x = x[['user_id', 'book_id_x', 'rank']]
        x = x.groupby('user_id').apply(lambda g: g.sort_values('rank', ascending=False)).reset_index(drop=True)

        return x

In [80]:
# Подготовка пайплайна
als_params = {
    'factors': 26,
    'iterations': 30,
    'random_state': 42
}

params_dict = {
        'seed': 42,
        'learning_rate': 0.1,
        'n_estimators': 1000,
        'max_depth': 6,
        'min_child_weight': 5,
        'gamma': 1,
        'device': 'cuda',
        'tree_method': 'hist'
    }

als_preprocessor = AlsFilter(als_params=als_params)
content_prep = ContentPreprocessor(books=books_df, genres=genres_df)
xgb_model = XGBRecomendation(xgb_params=params_dict)

pipe = Pipeline(steps=[
    ("als_preprocessor", als_preprocessor),
    ("content_preprocessor", content_prep),
    ("classifier", xgb_model)
])

In [ ]:
pipe.fit(train.drop(columns=['rating', 'book_order']), train['rating'])

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
preds = pipe.predict(user_sample)

C:\Users\Arsenij\AppData\Local\Temp\ipykernel_10724\4034111618.py:14: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  x = x.groupby('user_id').apply(lambda g: g.sort_values('rank', ascending=False)).reset_index(drop=True)


In [ ]:
scores = []
for user in preds.user_id.unique():
    recs = preds[preds['user_id'] == user].book_id_x.values
    recs = recs[:10]

    actual = test_sample[(test_sample.user_id == user) & (test_sample.rating==1)].book_id.tolist()
    score = apk_score(recs, actual, k=10)
    scores.append(score)

print('Итоговый показатель mAP@10: ', sum(scores) / len(scores))

Итоговый показатель mAP@10:  0.05635396825396825


Результат еще немного улучшился, но все еще далек от примлемого для допустимого к использованию в реальной ситуации данной рекомендательной системы.